### Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split

# Import uplift models
import torch
from catenets.models.torch import TARNet, SNet
from econml.metalearners import SLearner, XLearner
from econml.dr import LinearDRLearner
from econml.dml import CausalForestDML

# Datasets
from sklift.datasets import fetch_hillstrom

from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier

from scipy.stats import spearmanr



In [2]:
device = 'cpu'

### Helper functions

*METRICS*

In [3]:
import pandas as pd
import numpy as np

def safe_divide(numerator, denominator):
    return np.where(denominator != 0, numerator / denominator, 0)

def metrics_fast(df, perc=10):
    df = df.sort_values(by='ITE', ascending=False).reset_index(drop=True)
    
    # Convert relevant columns to NumPy arrays for faster processing
    obs = df['Obs'].values
    out = df['Out'].values
    not_obs = 1 - obs
    not_out = 1 - out

    # Calculate cumulative sums using NumPy
    cum_N1_T = np.cumsum(obs & out)
    cum_N0_T = np.cumsum(obs & not_out)
    cum_N1_C = np.cumsum(not_obs & out)
    cum_N0_C = np.cumsum(not_obs & not_out)

    N_1T = cum_N1_T[-1]
    N_0T = cum_N0_T[-1]
    N_1C = cum_N1_C[-1]
    N_0C = cum_N0_C[-1]
    
    N_T = N_1T + N_0T
    N_C = N_1C + N_0C
    L = len(df)

    Xphi = np.arange(L + 1) / L
    N1_T_over_N_T = N_1T / N_T
    N1_C_over_N_C = N_1C / N_C

    Qinilist = safe_divide(cum_N1_T, N_T) - safe_divide(cum_N1_C, N_C) - \
               (np.arange(1, L + 1) / L * (N1_T_over_N_T - N1_C_over_N_C))
    Qinilist = np.concatenate([[0], Qinilist])
    QS = np.trapz(Qinilist, Xphi)

    TOCY = safe_divide(cum_N1_T, cum_N1_T + cum_N0_T) - \
           safe_divide(cum_N1_C, cum_N1_C + cum_N0_C) + \
           N1_C_over_N_C - N1_T_over_N_T
    TOCY = np.concatenate([[0], TOCY])
    TOCS = np.trapz(TOCY, Xphi)

    ROCiniY = safe_divide(cum_N1_T, N_1T) - \
              safe_divide(cum_N0_T, N_0T) + \
              safe_divide(cum_N0_C, N_0C) - \
              safe_divide(cum_N1_C, N_1C)
    ROCiniY = np.concatenate([[0], ROCiniY])
    ROCiniS = np.trapz(ROCiniY, Xphi)

    pROCiniX = (safe_divide(cum_N0_T, N_0T) + safe_divide(cum_N1_C, N_1C)) / 2
    pROCiniX = np.concatenate([[0], pROCiniX])

    pROCiniY = (safe_divide(cum_N1_T, N_1T) + safe_divide(cum_N0_C, N_0C)) / 2
    pROCiniY = np.concatenate([[0], pROCiniY])

    pROCiniS = np.trapz(pROCiniY, pROCiniX)

    pTOCX = (safe_divide(cum_N1_C, cum_N1_C + cum_N0_C)) / N1_C_over_N_C 
    pTOCX = np.concatenate([[0], pTOCX])

    pTOCY = (safe_divide(cum_N1_T, cum_N1_T + cum_N0_T)) /N1_T_over_N_T
    pTOCY = np.concatenate([[0], pTOCY])

    pTOCS = np.trapz(pTOCY, pTOCX)






    CROCX = safe_divide(cum_N0_T + cum_N1_C, N_0T + N_1C)
    CROCX = np.concatenate([[0], CROCX])

    CROCY = safe_divide(cum_N1_T + cum_N0_C, N_1T + N_0C)
    CROCY = np.concatenate([[0], CROCY])

    CROCS = np.trapz(CROCY, CROCX)

    cutoff_index = int(L * perc / 100)
    df_top_perc = df.iloc[:cutoff_index]

    obs_top = df_top_perc['Obs'].values
    out_top = df_top_perc['Out'].values
    not_obs_top = 1 - obs_top
    not_out_top = 1 - out_top

    cum_N1_T_10 = np.cumsum(obs_top & out_top)
    cum_N0_T_10 = np.cumsum(obs_top & not_out_top)
    cum_N1_C_10 = np.cumsum(not_obs_top & out_top)
    cum_N0_C_10 = np.cumsum(not_obs_top & not_out_top)

    N_1T_10 = cum_N1_T_10[-1]
    N_0T_10 = cum_N0_T_10[-1]
    N_1C_10 = cum_N1_C_10[-1]
    N_0C_10 = cum_N0_C_10[-1]
    
    N_T_10 = N_1T_10 + N_0T_10
    N_C_10 = N_1C_10 + N_0C_10
    L_10 = len(df_top_perc)

    Xphi_10 = np.arange(L_10 + 1) / L_10
    N1_T_over_N_T_10 = safe_divide(N_1T_10, N_T_10)
    N1_C_over_N_C_10 = safe_divide(N_1C_10, N_C_10)
    Qinilist_10 = safe_divide(cum_N1_T_10, N_T_10) - \
                  safe_divide(cum_N1_C_10, N_C_10) - \
                  (np.arange(1, L_10 + 1) / L * (N1_T_over_N_T_10 - N1_C_over_N_C_10))
    Qinilist_10 = np.concatenate([[0], Qinilist_10])
    QS10 = np.trapz(Qinilist_10, Xphi_10)

    return QS10, QS, TOCS, ROCiniS, pROCiniS, CROCS, pTOCS




*SYNTHETIC DATA*

In [4]:
def generate_synthetic_outcomes(X, T, beta0=0.0, beta_coef=None, beta_t=0.5, sigma =0.1):
    """
    Generate synthetic probabilities and true individual treatment effects.
    The probability is defined as:
    
        p = 1 / (1 + exp( beta0 + X.dot(beta_coef) + beta_t * T) )
    
    where beta0 is the intercept, beta_coef are the coefficients for features in X,
    and beta_t is the treatment effect.
    
    The true individual treatment effect (ITE) is computed as the difference between the probability
    when T=1 and T=0.
    
    Returns:
        p: The computed probabilities.
        true_ITE: The true individual treatment effect.
    """
    if beta_coef is None:
        beta_coef = np.ones(X.shape[1])
    rnd_err = np.random.normal(scale= sigma)
    # Compute linear predictor for each unit (incorporating treatment effect)
    lin = beta0 + np.dot(X, beta_coef) + beta_t * T + rnd_err
    # Compute probability using a sigmoid of v * exp(linear predictor)
    p = 1 / (1 +  np.exp(-lin))
    
    # Compute true ITE: probability difference when T=1 vs T=0
    lin_treat = beta0 + np.dot(X, beta_coef) + beta_t + rnd_err # treatment scenario: T=1
    lin_control = beta0 + np.dot(X, beta_coef) + rnd_err           # control scenario: T=0
    p_treat = 1 / (1 + np.exp(-lin_treat))
    p_control = 1 / (1 + np.exp(-lin_control))
    true_ITE = p_treat - p_control
    
    return p, true_ITE, p_treat, p_control

In [5]:
# --- Define S-Learner functions ---
def train_predict_S_lr(X_train, T_train, Y_train, X_test):
       # Augment features by appending T as an extra column
       X_train_s = np.hstack([X_train, T_train.reshape(-1, 1)])
       model = LogisticRegression()
       model.fit(X_train_s, Y_train)
       # For test set, predict under treatment and control by replacing T with 1 and 0 respectively
       X_test_treated = np.hstack([X_test, np.ones((X_test.shape[0], 1))])
       X_test_control = np.hstack([X_test, np.zeros((X_test.shape[0], 1))])
       uplift = model.predict_proba(X_test_treated)[:, 1] - model.predict_proba(X_test_control)[:, 1]
       return uplift
       
def train_predict_S_xgb(X_train, T_train, Y_train, X_test):
       X_train_s = np.hstack([X_train, T_train.reshape(-1, 1)])
       model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
       model.fit(X_train_s, Y_train)
       X_test_treated = np.hstack([X_test, np.ones((X_test.shape[0], 1))])
       X_test_control = np.hstack([X_test, np.zeros((X_test.shape[0], 1))])
       uplift = model.predict_proba(X_test_treated)[:, 1] - model.predict_proba(X_test_control)[:, 1]
       return uplift
       
   # --- Define T-Learner functions ---
def train_predict_T_lr(X_train, T_train, Y_train, X_test):
       # Train separate models for treated and control samples
       model_treated = LogisticRegression()
       model_control = LogisticRegression()
       model_treated.fit(X_train[T_train == 1], Y_train[T_train == 1])
       model_control.fit(X_train[T_train == 0], Y_train[T_train == 0])
       uplift = model_treated.predict_proba(X_test)[:, 1] - model_control.predict_proba(X_test)[:, 1]
       return uplift
       
def train_predict_T_xgb(X_train, T_train, Y_train, X_test):
       model_treated = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
       model_control = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
       model_treated.fit(X_train[T_train == 1], Y_train[T_train == 1])
       model_control.fit(X_train[T_train == 0], Y_train[T_train == 0])
       uplift = model_treated.predict_proba(X_test)[:, 1] - model_control.predict_proba(X_test)[:, 1]
       return uplift

*EXPERIMENT*

In [6]:
def experiment_on_dataset(n = 1000):
    print("Running experiment on Hillstrom dataset")
    X, T = load_dataset()
    if not isinstance(X, np.ndarray):
        X = X.values
    # Split data into training and testing sets
    X_train, X_test, T_train, T_test = train_test_split(X, T, test_size=0.3, random_state=42)
    
    # Normalize training features and apply the same normalization to test features
    train_mean = X_train.mean(axis=0)
    train_std = X_train.std(axis=0)
    X_train = (X_train - train_mean) / train_std
    X_test = (X_test - train_mean) / train_std

    # Generate synthetic probabilities and true uplift for both sets
    p_train, true_ITE_train, _, _ = generate_synthetic_outcomes(X_train, T_train)
    p_test, true_ITE_test, _, _ = generate_synthetic_outcomes(X_test, T_test)
    
    # Plot histogram
    plt.hist(p_train, bins=30, density=True, edgecolor='k')
    plt.xlabel("Probability")
    plt.ylabel("Frequency")
    plt.title("Distribution of p_train")
    plt.show()
    
    # Print summary statistics
    # print("Mean:", np.mean(p_train))
    # print("Std:", np.std(p_train))
    # print("Min:", np.min(p_train))
    # print("Max:", np.max(p_train))

    # Collapse probabilities to discrete outcomes: 1 if p >= 0.5, else 0
    Y_train_GT = (p_train >= 0.5).astype(int)
    Y_test_GT = (p_test >= 0.5).astype(int)
    
    # Store our four models (two S-learners and two T-learners) in a dictionary
    GT_models = {
        'S_LR': train_predict_S_lr,
        'T_LR': train_predict_T_lr,
        'S_XGB': train_predict_S_xgb,
        'T_XGB': train_predict_T_xgb
    }
    #Get ground truth of models ranking

    # Create a dictionary to store the predicted ITEs and ranking scores for each model.
    GT_ranking_results = {}
    print(p_train)
    for name, func in GT_models.items():
        print(f"Training model: {name}")
        try:
            # Train model on (X_train, T_train, p_train) and get predicted uplift on X_test
            predicted_ITE = func(X_train, T_train, Y_train_GT, X_test)
        except Exception as e:
            print(f"Error training model {name}: {e}")
            predicted_ITE = np.zeros(len(T_test))
        
        # Compute a ranking distance metric between the predicted ITE ranking and the ground truth ranking.
        # Here we use Spearman's rank correlation coefficient.
        rho, _ = spearmanr(predicted_ITE, true_ITE_test)
        # A higher Spearman correlation means a better match.
        
        GT_ranking_results[name] = {
            'ranking_score': rho,
            'predicted_ITE': predicted_ITE
        }
        print(f"Ranking for {name}: Spearman rho = {rho:.4f}")
    # print(GT_ranking_results)

    # At this point, ranking_results contains each model's predicted ITEs and their ranking score.
    # Compute metrics for GT ranking.
    
    # Now draw binary outcomes based on the computed probabilities
    Y_train = np.random.binomial(1, p_train)
    trained_models_uplifts = {
        'S_LR': train_predict_S_lr,
        'T_LR': train_predict_T_lr,
        'S_XGB': train_predict_S_xgb,
        'T_XGB': train_predict_T_xgb
    }
        
    for name, func in trained_models_uplifts.items():
        print(f"Training model: {name}")
        try:
            # Train model on (X_train, T_train, p_train) and get predicted uplift on X_test
            uplift_pred = func(X_train, T_train, Y_train, X_test)
        except Exception as e:
            print(f"Error training model {name}: {e}")
            predicted_ITE = np.zeros(len(T_test))
        trained_models_uplifts[name] = uplift_pred

    results = {}
    for i in range(n):
        #sanity loop
        if i % (n/10) == 0:
            print("iteration:", i)
        Y_test = np.random.binomial(1, p_test)
        results_i = {}

        df_GT = pd.DataFrame({
            'Obs': T_test.astype(int),
            'Out': Y_test,
            'ITE': true_ITE_test
        })

        metrics_GT = metrics_fast(df_GT)
        results_i["GT"] = {
                'QS10': metrics_GT[0],
                'QS': metrics_GT[1],
                'TOCS': metrics_GT[2],
                'ROCiniS': metrics_GT[3],
                'pROCiniS': metrics_GT[4],
                'CROCS': metrics_GT[5],
                'pTOCS': metrics_GT[6]
            }
    
        for name, uplift_pred in trained_models_uplifts.items():
            # Prepare DataFrame for metric evaluation
            df_eval = pd.DataFrame({
                'Obs': T_test.astype(int),
                'Out': Y_test,
                'ITE': uplift_pred
            })
            metrics = metrics_fast(df_eval)
            results_i[name] = {
                'QS10': metrics[0],
                'QS': metrics[1],
                'TOCS': metrics[2],
                'ROCiniS': metrics[3],
                'pROCiniS': metrics[4],
                'CROCS': metrics[5],
                'pTOCS': metrics[6]
            }
            #print(f"Metrics for {name}: {results[name]}")
            results[i] = results_i
            
    return results, GT_ranking_results

In [7]:
def load_dataset(sample_size=1000):
    """
    Load the Hillstrom dataset from sklift.
    Returns features X (raw, with categorical variables converted to dummies)
    and a binary treatment indicator T.
    The treatment indicator is set to 1 if an E-Mail was sent ("Mens E-Mail" or "Womens E-Mail")
    and 0 if "No E-Mail".
    """
    dataset = fetch_hillstrom()
    df = dataset['data']
    if len(df) > sample_size:
        df = df.sample(sample_size, random_state=42)
        df = df.sample(sample_size)
    # Convert categorical features into dummy variables and cast to float
    X = pd.get_dummies(df, drop_first=True).astype(float)
    # Subsample the treatment indicator using the same indices as the data
    T = dataset['treatment'].loc[df.index]
    T = (T != "No E-Mail").astype(int).values
    return X, T

In [12]:
import numpy as np
from scipy.stats import spearmanr, kendalltau, wasserstein_distance
def weighted_kendall_tau(x, y, w=None):
    n = len(x)
    assert len(y) == n
    if w is None:
        w = np.abs(np.subtract.outer(y, y))
    num = 0.0
    denom = 0.0
    for i in range(n):
        for j in range(i+1, n):
            a = np.sign(x[i] - x[j])
            b = np.sign(y[i] - y[j])
            weight = w[i, j]
            num += weight * (a != b)
            denom += weight
    return 1 - (num / denom) if denom != 0 else np.nan



def prepare_dataset_and_predict_ites():
    print("Running experiment on Hillstrom dataset")
    
    X, T = load_dataset()
    if not isinstance(X, np.ndarray):
        X = X.values
    
    X_train, X_test, T_train, T_test = train_test_split(X, T, test_size=0.3, random_state=42)

    train_mean = X_train.mean(axis=0)
    train_std = X_train.std(axis=0)
    X_train = (X_train - train_mean) / train_std
    X_test = (X_test - train_mean) / train_std

    p_train, true_ITE_train, _, _ = generate_synthetic_outcomes(X_train, T_train)
    p_test, true_ITE_test, p_treat_test, p_control_test = generate_synthetic_outcomes(X_test, T_test)

    Y_train_GT = (p_train >= 0.5).astype(int)

    GT_models = {
        'S_LR': train_predict_S_lr,
        'T_LR': train_predict_T_lr,
        'S_XGB': train_predict_S_xgb,
        'T_XGB': train_predict_T_xgb
    }

    predicted_ites_dict = {}
    for name, func in GT_models.items():
        print(f"Training model: {name}")
        try:
            predicted_ITE = func(X_train, T_train, Y_train_GT, X_test)
        except Exception as e:
            print(f"Error training model {name}: {e}")
            predicted_ITE = np.zeros(len(T_test))
        predicted_ites_dict[name] = predicted_ITE

    df_ITES = pd.DataFrame(predicted_ites_dict)
    df_ITES["true_ITE"] = true_ITE_test
    df_ITES["PT"] = p_treat_test
    df_ITES["PC"] = p_control_test


    # Uniform histogram settings
    bins = 40
    min_val = df_ITES.min().min()
    max_val = df_ITES.max().max()
    y_max = 0
    hist_data = {}
    model_cols = [col for col in df_ITES.columns if col not in {"PT", "PC"}]
    for col in model_cols:
        counts, bin_edges = np.histogram(df_ITES[col], bins=bins, range=(min_val, max_val), density=True)
        hist_data[col] = counts
        y_max = max(y_max, counts.max())
    for col in model_cols:
        plt.hist(df_ITES[col], bins=bins, range=(min_val, max_val), density=True, edgecolor='k')
        plt.title(f"Distribution of {col}")
        plt.xlabel("ITE")
        plt.ylabel("Density")
        plt.xlim(min_val, max_val)
        plt.ylim(0, y_max * 1.05)
        plt.show()

    # Metric computation
    results = {}
    for col in model_cols:
        if col == "true_ITE":
            continue
        x = df_ITES[col].values
        y = df_ITES["true_ITE"].values

        mse = np.mean((x - y) ** 2)
        rho, _ = spearmanr(x, y)
        tau, _ = kendalltau(x, y)
        w_kendall = weighted_kendall_tau(x, y)
        emd = wasserstein_distance(x, y)

        rank_x = pd.Series(x).rank().values
        rank_y = pd.Series(y).rank().values
        mean_rank_dist = np.mean(np.abs(rank_x - rank_y))

        results[col] = {
            'MSE': mse,
            'Spearman_rho': rho,
            'Kendall_tau': tau,
            'Weighted_Kendall': w_kendall,
            'EMD': emd,
            'Mean_Rank_Dist': mean_rank_dist
        }

    df_metrics = pd.DataFrame(results).T

    def color_ranks(s, maximize=True):
        order = s.rank(ascending=not maximize, method="min")
        colors = []
        for r in order:
            if r == 1:
                colors.append('background-color: #228B22')  # light green
            elif r == 2:
                colors.append('background-color: #1E90FF')  # light blue
            elif r == 3:
                colors.append('background-color: #9370DB')  # light purple
            else:
                colors.append('')
        return colors

    styled = (
        df_metrics.style
        .apply(color_ranks, subset=['Spearman_rho', 'Kendall_tau', 'Weighted_Kendall'], maximize=True)
        .apply(color_ranks, subset=['MSE', 'EMD', 'Mean_Rank_Dist'], maximize=False)
        .format(precision=4)
    )
    
    display(styled)
    display(df_ITES)
    return df_ITES


In [ ]:
df=prepare_dataset_and_predict_ites()

In [ ]:
df

In [ ]:
last_two_columns = df.iloc[:, -2:]

# Count zeros and ones
count_zeros = (last_two_columns <0.01).sum().sum()
count_ones = (last_two_columns >0.99).sum().sum()

print("Number of zeros in last 2 columns:", count_zeros)
print("Number of ones in last 2 columns:", count_ones)

In [ ]:
df

In [184]:
from tqdm import tqdm
def simulate_and_score_uplift_metrics(df_ITES, n_runs=1, verbose=True):
    """
    Runs the uplift simulation and evaluation n_runs times.
    Returns:
        metrics_dict: {model_name: {metric_name: [v1, v2, ..., v_n_runs], ...}, ...}
    """
    uplift_cols = ['S_LR', 'T_LR', 'S_XGB', 'T_XGB', 'true_ITE']
    metric_names = ['QS10', 'QS', 'TOCS', 'ROCiniS', 'pROCiniS', 'CROCS', 'pTOCS']

    # Initialize result structure
    metrics_dict = {
        model: {metric: [] for metric in metric_names}
        for model in uplift_cols
    }

    for run in tqdm(range(n_runs)):
        N = len(df_ITES)
        PT = df_ITES["PT"].values
        PC = df_ITES["PC"].values

        Obs = np.random.binomial(1, 0.5, N)
        OutT = np.random.binomial(1, PT)
        OutC = np.random.binomial(1, PC)
        Out = np.where(Obs == 1, OutT, OutC)

        df_aug = df_ITES.copy()
        df_aug["Obs"] = Obs
        df_aug["OutT"] = OutT
        df_aug["OutC"] = OutC
        df_aug["Out"] = Out

        for col in uplift_cols:
            df_eval = df_aug[['Obs', 'Out', col]].copy()
            df_eval.columns = ['Obs', 'Out', 'ITE']

            try:
                with np.errstate(divide='ignore', invalid='ignore'):
                    QS10, QS, TOCS, ROCiniS, pROCiniS, CROCS, _ = metrics_fast(df_eval)
                    values = [QS10, QS, TOCS, ROCiniS, pROCiniS, CROCS]
            except Exception as e:
                if verbose:
                    print(f"[ERROR] Run {run}, model {col}")
                    print(df_eval.head())
                    import traceback
                    traceback.print_exc()
                values = [np.nan] * 7

            for name, val in zip(metric_names, values):
                metrics_dict[col][name].append(val)

    return metrics_dict


In [ ]:
metrics_list= simulate_and_score_uplift_metrics(df,1000000)


In [ ]:
metrics_list

In [203]:
def evaluate_ordering_consistency_table(metrics_dict):
    """
    Evaluates how often the expected pairwise ordering holds across n_runs.

    Input:
        metrics_dict: nested dict of form
            { model: { metric: [v1, ..., vn] } }

    Output:
        DataFrame with metrics as rows and columns:
            'true_ITE > S_LR',
            'S_LR > S_XGB',
            'S_XGB > T_LR',
            'T_LR > T_XGB'
        Values are counts over n_runs
    """
    expected_order = ['true_ITE', 'S_LR',  'T_LR','S_XGB', 'T_XGB']
    comparisons = [
        ('true_ITE', 'S_LR'),
        ('S_LR', 'T_LR'),
        ('T_LR','S_XGB'),
        ('S_XGB', 'T_XGB')
    ]
    comparison_labels = [
        'true_ITE > S_LR',
        'S_LR > T_LR',
        'T_LR > S_XGB ',
        'S_XGB > T_XGB'
    ]

    metric_names = list(next(iter(metrics_dict.values())).keys())
    n_runs = len(next(iter(metrics_dict.values())).values().__iter__().__next__())
    counts = {metric: [0, 0, 0, 0] for metric in metric_names}

    for metric in metric_names:
        for i, (a, b) in enumerate(comparisons):
            a_vals = metrics_dict[a][metric]
            b_vals = metrics_dict[b][metric]
            for va, vb in zip(a_vals, b_vals):
                if not (np.isnan(va) or np.isnan(vb)) and va > vb:
                    counts[metric][i] += 1


    df_result = pd.DataFrame.from_dict(counts, orient='index', columns=comparison_labels) / n_runs * 100

    return df_result


In [226]:
df_ordering = evaluate_ordering_consistency_table(metrics_list)

In [ ]:
df_ordering